In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa # soundfile 대신 librosa 사용 (MP3 지원 및 리샘플링에 용이)
import torch
# from jiwer import wer # 현재 코드에서는 사용되지 않음

# MPS 장치 설정 또는 CPU 사용 (MPS 사용에 문제 없으면 그대로 두셔도 됩니다)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# 프로세서 및 모델 로드
processor_name = "kresnik/wav2vec2-large-xlsr-korean"
model_name = "kresnik/wav2vec2-large-xlsr-korean"

try:
    processor = Wav2Vec2Processor.from_pretrained(processor_name)
    model = Wav2Vec2ForCTC.from_pretrained(model_name,torch_dtype=torch.bfloat16).to(device)
except Exception as e:
    print(f"모델 또는 프로세서 로딩 중 오류 발생: {e}")
    exit()

# 오디오 파일 경로
audio_file_path = "/Users/gim-yonghyeon/Documents/GitHub/stt_project/일본어+한국어.mp3"
target_sampling_rate = 16000 # Wav2Vec2 모델의 일반적인 샘플링 레이트

try:
    # librosa를 사용하여 오디오 파일 로드 및 리샘플링
    # sr=None으로 설정하면 원본 샘플링 레이트로 로드하지만, Wav2Vec2는 특정 SR을 기대하므로 target_sampling_rate로 지정
    speech_array, original_sampling_rate = librosa.load(audio_file_path, sr=target_sampling_rate, mono=True)
    print(f"오디오 로드 완료. 샘플링 레이트: {target_sampling_rate}Hz, 오디오 길이 (샘플 수): {len(speech_array)}")
except Exception as e:
    print(f"오디오 파일 로딩 또는 리샘플링 중 오류 발생: {e}")
    exit()

# 오디오 데이터 전처리 (프로세서에 샘플링 레이트 명시)
# input_values = processor(speech_array, sampling_rate=target_sampling_rate, return_tensors="pt", padding=True).input_values.to(device)
# 위 방식이 일반적이나, input_features를 사용하라는 경고가 있었으므로 명시적으로 사용해볼 수 있습니다.
# 다만, Wav2Vec2Processor의 __call__은 내부적으로 feature_extractor를 호출하므로 sampling_rate를 넘기는 것이 중요합니다.
features = processor(speech_array, sampling_rate=target_sampling_rate, return_tensors="pt", padding=True)
input_values = features.input_values.to(dtype=torch.bfloat16).to(device)
# attention_mask = features.attention_mask.to(device) # 필요한 경우 사용

# 음성 인식 실행
with torch.no_grad():
    logits = model(input_values).logits # attention_mask도 전달 가능: model(input_values, attention_mask=attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print("인식 결과:", transcription)

Using device: mps
오디오 로드 완료. 샘플링 레이트: 16000Hz, 오디오 길이 (샘플 수): 1489607
인식 결과: 민화상 니지 영 새 한국서르더 간나분아이 무 강토 그리니 고이라노 크리니니 렌케해 트 적세에 다트후 당세에 다트후 어 새후했더 모 하지후 돌아부룬이 나향데일 하시까마다 터스바야 미워인노스  주오 캐달해도 카다나도 가실 킬어오 격사문 아니라 사이 모치럼 림포가람 먹고이 그다 사는 오대으나 츠와이 의원 강부그리니까 한 부 고할이 크트 진당지 캐르 고도 닫되 기는데 너더 그리니 이 운 것에 어 나힘 야거이라 토오르크시데이튼나기 서 단나 이어 어그 그다 먼다인후무려전후고 인 정 타지려 후 지료 휘나 더니 지 때 것시라 살시만 지 캐아이그 바이대테이 야그 이 현물어 그리이지만 민너성카라 노치킬더워 그데이 터나끼   트도 거서 단 후지려 스 슴에 때마 이 하나님가 페입판에로 거리워의 된다게이 의원 감법워 그니까 우리 시오리 큰부니었를지거서 단달더 기가로니거리나  크다사다
